In [2]:
import pandas as pd 
import numpy as np

In [3]:
X_train=pd.read_csv("Deep_learning_Scratch/mnist_train.csv")

In [12]:
Y_train=X_train.iloc[:,0]
X_train=X_train.iloc[:,1:]

In [13]:
X_train.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.608,0.609,0.610,0.611,0.612,0.613,0.614,0.615,0.616,0.617
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
X_train.shape,Y_train.shape

((59999, 784), (59999,))

In [15]:
Y_train.head()

0    0
1    4
2    1
3    9
4    2
Name: 5, dtype: int64

In [75]:
import numpy as np

class ANN:
    def __init__(self):
        self.W1 = []
        self.W2 = []
        self.b1 = []
        self.b2 = []
        self.input_n = 784

    def relu(self, Z):
        return np.maximum(0, Z)
    
    def softmax(self, Z):
        exp_z = np.exp(Z - np.max(Z, axis=1, keepdims=True))  # for batch stability
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def initialize_weights_and_biases(self, input_dim, hidden_dim, output_dim):
        self.W1 = np.random.uniform(low=-1, high=1, size=(input_dim, hidden_dim))
        self.b1 = np.random.uniform(low=-1, high=1, size=(hidden_dim,))

        self.W2 = np.random.uniform(low=-1, high=1, size=(hidden_dim, output_dim))
        self.b2 = np.random.uniform(low=-1, high=1, size=(output_dim,))

    def forward_prop(self, X):
        Z1 = np.dot(X, self.W1) + self.b1  # bias added
        A1 = self.relu(Z1)
        Z2 = np.dot(A1, self.W2) + self.b2 # bias added
        A2 = self.softmax(Z2)
        return A2
    
    def get_class_labels(self, A2):
        return np.argmax(A2, axis=1)  # argmax over classes for each sample

    def one_hot_encoding(self, A2):
        class_labels = self.get_class_labels(A2)
        one_hot_y = np.zeros_like(A2)
        one_hot_y[np.arange(len(class_labels)), class_labels] = 1
        return one_hot_y


        

In [72]:
obj=ANN()
obj.initialize_weights_and_biases(784,10,10)

(59999, 10)